The stack is for small immutable concrete objects.
The compiler knows their current and later states.
The stack objects are fast to work with.

The heap is for all other objects.
These objects have unknown types, sizes, fields, or other attributes and can even change to something else.
The heap objects are slow to work with.

One allocation means that one object was born in the heap.

In [15]:
# These objects are hard-coded (immutable) and concrete.

@btime 1

@btime 'a'

@btime "a"

@btime ()

;

  1.208 ns (0 allocations: 0 bytes)
  1.208 ns (0 allocations: 0 bytes)
  2.125 ns (0 allocations: 0 bytes)
  1.166 ns (0 allocations: 0 bytes)


In [16]:
# These objects are not concrete.

@btime []

@btime Set()

@btime Dict()

;

  16.074 ns (1 allocation: 48 bytes)
  57.986 ns (4 allocations: 384 bytes)
  58.325 ns (4 allocations: 512 bytes)


In [22]:
va = 1

const vac = 1

;

In [46]:
# These containers have known types but not size, so they are not concrete and are born in the heap.

@btime Char[]

@btime Set{Float64}()

@btime Dict{String, Int}()

;

  16.784 ns (1 allocation: 64 bytes)
  90.168 ns (4 allocations: 400 bytes)
  58.494 ns (4 allocations: 528 bytes)


In [47]:
# All the values are born in the stack but the containers difffer.

@btime (1,2,3,4,5,6,7,8,9,10)

@btime [1,2,3,4,5,6,7,8,9,10]

;

  1.166 ns (0 allocations: 0 bytes)
  17.743 ns (1 allocation: 144 bytes)


In [43]:
# Using the mutable version allocates `Any[]` and then, after infering its type during run time, `Bool[]`.

@btime [nu == va for nu in (1,2,3)]

@btime [nu == vac for nu in (1,2,3)]

;

  416.246 ns (2 allocations: 96 bytes)
  17.493 ns (1 allocation: 64 bytes)


Profile allocations to see this temporary `Any[]` allocation.

In [44]:
function fu()
    
    [nu == va for nu in (1,2,3)]
    
end

function fuc()
    
    [nu == vac for nu in (1,2,3)]
    
end

fuc (generic function with 1 method)

In [42]:
@btime fu()

@btime fuc()

;

  414.362 ns (2 allocations: 96 bytes)
  17.452 ns (1 allocation: 64 bytes)


In [37]:
Profile.Allocs.clear()

Profile.Allocs.@profile sample_rate=1 fu()

PProf.Allocs.pprof(from_c=false)

"alloc-profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:62261


In [39]:
Profile.Allocs.clear()

Profile.Allocs.@profile sample_rate=1 fuc()

PProf.Allocs.pprof(from_c=false)

"alloc-profile.pb.gz"

Main binary filename not available.
Serving web UI on http://localhost:62261
